# 1.存储数据


In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import sys
import os
sys.path.append(os.path.abspath('../..'))  # 添加项目根目录到Python路径


from dotenv import load_dotenv

load_dotenv()

True

## 数据连接

In [2]:
import pymysql
import os

def create_connection():
    """
    创建并返回一个数据库连接。
    
    Returns:
        connection: 数据库连接对象，如果连接失败则返回 None。
    """
    sql_host = os.getenv("MYSQL_HOST")
    sql_user = os.getenv("MYSQL_USERNAME")
    sql_password = os.getenv("MYSQL_PASSWORD")
    sql_db = os.getenv("MYSQL_DB")
    sql_charset = os.getenv("MYSQL_CHARSET")
    
    try:
        connection = pymysql.connect(
            host=sql_host,
            user=sql_user,
            password=sql_password,
            database=sql_db,
            charset=sql_charset
        )
        print("数据库连接成功！")
        return connection.cursor()
    except pymysql.InterfaceError as e:
        print(f"数据库连接错误: {e}")
        return None
    except Exception as e:
        print(f"发生错误: {e}")
        return None

my_sql = create_connection()

数据库连接成功！


In [4]:
from research_agent.core.general_llm import LLM
from research_agent.core.config import Config
from pyaml_env import parse_config

configs = parse_config(Config.YAML_CONFIG)


# deepseek_llm = LLM(config=configs["deepseek-chat"])

## 专利申请人排名数据查询

In [ ]:
# 希望在数据存入库中的时候，能够把专利申请人分开

In [68]:
def get_applicant_data(top_n=5):
    my_sql = create_connection()
    query = """
    SELECT 
        `申请人`,
        -- 合并格式：中国(32), 美国(5)
        GROUP_CONCAT(
            CONCAT(`国家`, '(', `分国专利数`, ')') 
            SEPARATOR ', '
        ) AS `受理局及数量`,
        CAST(SUM(`分国专利数`) AS UNSIGNED) AS `总专利数`  -- 转为整数
    FROM (
        -- 先按申请人和受理局分组，计算分国专利数
        SELECT 
            `[标]当前申请(专利权)人` AS `申请人`,
            `受理局` AS `国家`,
            CAST(COUNT(*) AS UNSIGNED) AS `分国专利数`  -- 转为整数
        FROM 
            `patent_table`
        WHERE 
            `[标]当前申请(专利权)人` IS NOT NULL
        GROUP BY 
            `申请人`, `国家`
    ) AS subquery
    GROUP BY 
        `申请人`
    ORDER BY 
        `总专利数` DESC;
        """
    my_sql.execute(query=query)
    result = my_sql.fetchall()
    # 获取排名前五的机构的专利信息：公开号、标题、摘要、申请人
    applicants = [x for x,y,z in result[:top_n]]
    placeholders = ','.join(['%s'] * len(applicants))
    query = f"""
    SELECT `公开(公告)号`, `标题(译)(简体中文)`, `摘要(译)(简体中文)`,`[标]当前申请(专利权)人`
    FROM `patent_table`
    WHERE `[标]当前申请(专利权)人` IN ({placeholders})
    ORDER BY `[标]当前申请(专利权)人`
    """

    my_sql.execute(query, applicants)
    applicant_rank = result[:top_n]
    applicant_data = my_sql.fetchall()
    return applicant_rank,applicant_data


In [69]:
applicant_rank,applicant_data = get_applicant_data()

数据库连接成功！


In [70]:
applicant_rank

(('杭州富加镓业科技有限公司', '中国(32), 日本(2), 美国(1)', 35),
 ('信越化学工业株式会社', '韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)', 29),
 ('日本碍子株式会社', '日本(10), 美国(2), 世界知识产权组织(4), 中国(7)', 23),
 ('中国电子科技集团公司第四十六研究所', '中国(18)', 18),
 ('山东大学', '中国(17), 日本(1)', 18))

## 专利申请人-摘要— 专利布局分析

In [7]:
map_tech = '''
### 制备技术
    - 单晶生长技术
    - 衬底加工技术
    - 外延生长、薄膜制备技术
    - 器件工艺
### 器件及应用
    - 日盲紫外光电探测器
    - 红外焦平面阵列
    - X射线探测器
    - 气敏传感器
    - 功率电子器件
'''

In [ ]:
from llama_index.llms.deepseek import DeepSeek

# you can also set DEEPSEEK_API_KEY in your environment variables
llm = DeepSeek(model="deepseek-chat", api_key="sk-b37943bec0a845ed983a2104f09d2d8b")


In [9]:
import json_repair
from research_agent.core.general_llm import LLM
configs = parse_config(Config.YAML_CONFIG)
deepseek_llm = LLM(config=configs["deepseek-chat"])


In [81]:

async def  analysis_classification(company_name,data):
    # 初始化结果字符串列表
    result_strings = []
    
    # 遍历数据，构建专利信息字符串
    for idx, row in data.iterrows():
        # 使用f-string格式化每个专利的信息
        patent_info = f"id:{row['id']};专利标题：{row['title']}；专利摘要：{row['abstract']}"
        result_strings.append(patent_info)
    prompt_a = [{"role": "user", 
                 "content": f"""
                **角色**：资深半导体技术专利分析师，擅长技术路线图绘制与创新点挖掘
                **输入数据**：以下专利申请人{company_name}的{len(result_strings)}项专利数据（编号0-{len(result_strings)}）：
                {result_strings}

                **核心任务**：
                1. 专利分类（确保处理所有编号）：
                    - ###为一级分类；-为二级分类
                    - 以下是包含具体的一级分类及二级分类的**技术图谱**
                        ```markdown
                        {map_tech}
                        ```
                    - 每项专利必须输出["一级分类", "二级分类"]
                    - 输出的一级分类、二级分类需要严格按照**技术图谱**中的命名

                2. **综合技术挖掘**（基于所有专利）：
                    ■ 核心技术方向：提炼专利共同聚焦的3-5个技术方向, 
                    ■ 技术问题解决分析：
                        - 针对哪些技术痛点，提出了哪些解决方案，效果指标如何
                    ■ 典型案例说明：
                        - 选择出具有代表性的3-5篇专利，论证这些专利是如何实现对技术问题的解决的。

                **处理规则**：
                    1. 分类阶段逐项处理，输出后需验证："已确认处理0-{len(result_strings)}号专利"
                    2. 技术挖掘必须引用具体专利内容，禁止臆测
                    3. 请输出有效的json格式
                    4. 综合技术挖掘的分析需要
                **输出格式**：
                    ```json
                    {{
                    "分类结果":{{"id_num":["一级分类","二级分类"],}}
                    "验证状态":"完成多少项分类",
                    "综合技术挖掘"："在这里写下你基于所有专利进行的**综合技术挖掘**"
                    }}
                    ```
                **输出示例**：
                    ```json
                    {{
                    "分类结果": {{
                        "CN.....A": ["制备技术", "外延生长技术"],
                        "CN.....A": ["制备技术", "晶体生长设备"],
                        ...}}，
                    "验证状态": "完成2/2项分类",
                    "综合技术挖掘":"均胜公司在车用气囊气体发生器研制方面经验愈发丰富，创新成果不断推出，专利申请量排位第一。其专利主要通过对点火组件、药剂存储装置、密封防潮结构、排气组件、过滤系统等进行优化设计，提升了气囊的充气速度、稳定性及安全性等。如在专利CN210591781U中提出了一种混合式气体发生器排气装置，有效避免了密封膜在气流作用下脱落，及时扩大排气孔孔径，恢复气流通路，确保安全气囊的正常工作；在专利CN116160991A中提出了一种新型气帘用混合式气体发生器，有效防止传火药吸湿受潮，保证气体发生器的功能正常，确保气帘的安全性和可靠性；在专利CN217259932U中公开了一种免焊接型安全气囊用气体发生器，降低了生产成本，简化了结构，提高了产品的安全性和密封性能。"
                    }}
                    ```
                """}]
    result = await deepseek_llm.completion(prompt_a)
    result_json = json_repair.loads(result)
    result_json["company_name"] = company_name
    return result_json


In [ ]:
import pandas as pd
df = pd.DataFrame(applicant_data,columns=["id","title","abstract","company"])
df

In [83]:
analysis_task = [analysis_classification(a,df[df["company"]==a]) for a,_,x in applicant_rank]
# analysis_task.append(analys_zhuti(applicant_data))
analysis_result = await asyncio.gather(*analysis_task)

{'model': 'deepseek-chat', 'messages': [{'role': 'user', 'content': '\n                **角色**：资深半导体技术专利分析师，擅长技术路线图绘制与创新点挖掘\n                **输入数据**：以下专利申请人杭州富加镓业科技有限公司的35项专利数据（编号0-35）：\n                [\'id:CN113913924A;专利标题：一种氧化镓单晶生长装置；专利摘要：本发明公开了一种氧化镓单晶生长装置，其包括氧化镓晶体生长炉以及设置在所述氧化镓晶体炉侧边的激光辅助加热器，所述激光辅助加热器发出的激光射入所述氧化镓晶体生长炉内部。本发明通过引入激光辅助加热器，利用激光定向精准加热功能，使模具口附近的晶体生长固液界面处的过冷区域快速局部升温，可以在很短的时间内消除过冷现象，从而确保高质量氧化镓晶体的顺利生长。\', \'id:CN215976140U;专利标题：一种螺旋式籽晶杆；专利摘要：本实用新型公开一种螺旋式籽晶杆，其包括上直杆、下直杆以及位于所述上直杆和所述下直杆之间的螺旋杆。本实用新型创新性地提出了在上直杆和所述下直杆之间设置螺旋杆，通过所述螺旋杆来吸附晶体生长过程中的挥发物，可以有效的吸附从保温材料的内部通腔向上扩散以及向下飘落的挥发物，降低挥发物附着到晶体生长固液界面和观察窗上的概率，提高氧化镓单晶生长的成功率和晶体生长的实时观测效果。\', \'id:CN215976141U;专利标题：一种用于生长氧化镓晶体的籽晶杆；专利摘要：本实用新型公开一种用于生长氧化镓晶体的籽晶杆，其包括中心杆以及环绕设置在所述中心杆周边的遮挡片组件。本实用新型创新性的提出了在中心杆上安装遮挡片组件来吸附晶体生长过程中的氧化镓挥发物，可以有效的吸附从保温材料的内部通腔向上扩散以及向下飘落的氧化镓挥发物，降低氧化镓挥发物附着到晶体生长固液界面和观察窗上的概率，提高氧化镓单晶生长的成功率和晶体生长的实时观测效果。\', \'id:CN114908418A;专利标题：一种在导电型氧化镓衬底上制备同质外延氧化镓薄膜的方法及HVPE设备；专利摘要：本发明提供了一种在导电型氧化镓衬底上制备同质外延氧化镓薄膜的方法及HVPE设备，所述HVPE设备包括HVPE反应室、设置在所述HV

In [ ]:
df[df["company"]=="杭州富加镓业科技有限公司"]

In [84]:
analysis_result

[{'分类结果': {'CN113913924A': ['制备技术', '单晶生长技术'],
   'CN215976140U': ['制备技术', '单晶生长技术'],
   'CN215976141U': ['制备技术', '单晶生长技术'],
   'CN114908418A': ['制备技术', '外延生长、薄膜制备技术'],
   'CN114908419B': ['制备技术', '外延生长、薄膜制备技术'],
   'CN113957519B': ['制备技术', '单晶生长技术'],
   'CN113957529A': ['制备技术', '单晶生长技术'],
   'CN113957518A': ['制备技术', '单晶生长技术'],
   'CN113026103B': ['制备技术', '单晶生长技术'],
   'CN117626408A': ['制备技术', '单晶生长技术'],
   'CN113913925A': ['制备技术', '单晶生长技术'],
   'CN114086254B': ['制备技术', '单晶生长技术'],
   'CN117552092A': ['制备技术', '单晶生长技术'],
   'CN114086244A': ['制备技术', '单晶生长技术'],
   'CN117867636A': ['制备技术', '单晶生长技术'],
   'CN117976518A': ['制备技术', '外延生长、薄膜制备技术'],
   'CN114134561A': ['制备技术', '单晶生长技术'],
   'CN114808118A': ['制备技术', '外延生长、薄膜制备技术'],
   'CN214458441U': ['制备技术', '单晶生长技术'],
   'CN114959868A': ['制备技术', '单晶生长技术'],
   'CN112853470A': ['制备技术', '单晶生长技术'],
   'CN112859771A': ['制备技术', '单晶生长技术'],
   'CN112863620A': ['制备技术', '单晶生长技术'],
   'CN112837758A': ['制备技术', '单晶生长技术'],
   'CN112831832A': ['制备技术', '单晶生长技术'

In [85]:
applicant_rank

(('杭州富加镓业科技有限公司', '中国(32), 日本(2), 美国(1)', 35),
 ('信越化学工业株式会社', '韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)', 29),
 ('日本碍子株式会社', '日本(10), 美国(2), 世界知识产权组织(4), 中国(7)', 23),
 ('中国电子科技集团公司第四十六研究所', '中国(18)', 18),
 ('山东大学', '中国(17), 日本(1)', 18))

In [86]:
# 将分析结果保存为JSON文件
import json

# 将分析结果转换为字典格式
result_dict = {r["company_name"]: r["分类结果"] for r in analysis_result}

# 保存为JSON文件
with open("analysis_results.json", "w", encoding="utf-8") as f:
    json.dump(analysis_result, f, ensure_ascii=False, indent=4)

# 打印保存成功信息
print("分析结果已保存到 analysis_results.json 文件中")


分析结果已保存到 analysis_results.json 文件中


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter, defaultdict
import matplotlib
# 设置中文字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体为黑体
matplotlib.rcParams['axes.unicode_minus'] = False  # 解决保存图像时负号'-'显示为方块的问题

## web_search：申请人-技术布局


In [ ]:
# import json
# with open("analysis_results.json", 'r', encoding='utf-8') as f:
#     result_dict = json.load(f)
# result_dict


In [87]:
result_dict = {r["company_name"]: r["分类结果"] for r in analysis_result}
result_dict
company_tech = {}
for x,y in result_dict.items():
    print(x)
    tech = set()
    for _, yy in y.items():
        tech.add(yy[1])
    company_tech[x] = list(tech)
company_tech

杭州富加镓业科技有限公司
信越化学工业株式会社
日本碍子株式会社
中国电子科技集团公司第四十六研究所
山东大学


{'杭州富加镓业科技有限公司': ['单晶生长技术', '外延生长、薄膜制备技术'],
 '信越化学工业株式会社': ['外延生长、薄膜制备技术', '衬底加工技术'],
 '日本碍子株式会社': ['单晶生长技术', '外延生长、薄膜制备技术', '衬底加工技术'],
 '中国电子科技集团公司第四十六研究所': ['单晶生长技术', '外延生长、薄膜制备技术', '衬底加工技术'],
 '山东大学': ['外延生长、薄膜制备技术', '日盲紫外光电探测器', '单晶生长技术', '红外焦平面阵列', '功率电子器件']}

In [ ]:
from research_agent.core.general_llm import LLM
from research_agent.core.config import Config
configs = parse_config(Config.YAML_CONFIG)
search_llm = LLM(config=configs["glm-4"])
async def search_applicants(company_name,domain):
    # 定义工具参数
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True,
            "search_engine": "search_std",  # 选择搜索引擎类型，默认为基础版
            "search_result": True,
            "search_prompt": "你是一名专利分析师，请用简洁的语言总结网络搜索中：{{search_result}}中的关键信息"
        }
    }]

    # 定义用户消息
    messages = [{
        "role": "user",
        "content": f"""
        100字简要介绍{company_name}在{domain}上的技术布局和发展
        """
    }]

    result = await search_llm.completion(messages=messages,tools=tools)
    # 打印响应结果
    return result

search_task = [search_applicants(company_name,domain) for company_name,domain in company_tech.items()]
search_result = await asyncio.gather(*search_task)
search_result

In [ ]:
import asyncio
from pyaml_env import parse_config
from research_agent.core.general_llm import LLM
configs = parse_config(
    r"D:\GoodStudy\AI_Agents_Hackathon\awesome_patent_mining\patent_analysis_agent\research_agent\core\llm_config.yaml")
print(configs["glm-4"])
# 定义工具参数示例
tools = [{
    "type": "web_search",
    "web_search": {
        "enable": True,
        "search_engine": "search_std",  # 选择搜索引擎类型，默认为基础版
        "search_result": True,
        "search_prompt": "你是一名专利分析师，请用简洁的语言总结网络搜索中：{{search_result}}中的关键信息"
    }
}]

# 定义用户消息
messages = [{
    "role": "user",
    "content": f"""
    100字简要介绍北京铭镓半导体有限公司在第三代半导体材料制备上的技术布局和发展\n标注最新信息时间        
    """
}]

llm = LLM(config=configs["glm-4"])
# messages = [{"role": "user", "content": "\n        机构名称：杭州富加镓业科技有限公司\n        领域方向：氧化镓材料\n        使用`web_search`工具检索相关内容，\n        100字简要介绍该机构在领域方向上的技术布局和发展\n        "}]

print(asyncio.run(llm.completion(messages, tools=tools)))  # 使用工具

## 专利-技术 布局分析

In [20]:
result_dict = {r["company_name"]: r["分类结果"] for r in analysis_result}
result_dict

{'杭州富加镓业科技有限公司': {'CN112831832A': ['制备技术', '单晶生长技术'],
  'CN113026103B': ['制备技术', '单晶生长技术'],
  'CN215976140U': ['制备技术', '单晶生长技术'],
  'CN117976518A': ['制备技术', '外延生长、薄膜制备技术'],
  'US12024791B2': ['制备技术', '单晶生长技术'],
  'JP2023512384A': ['制备技术', '单晶生长技术'],
  'JP7478817B2': ['制备技术', '单晶生长技术'],
  'CN112877770A': ['制备技术', '单晶生长技术'],
  'CN215976139U': ['制备技术', '单晶生长技术'],
  'CN215976134U': ['制备技术', '单晶生长技术'],
  'CN112820360A': ['制备技术', '单晶生长技术'],
  'CN117552092A': ['制备技术', '单晶生长技术'],
  'CN112837758A': ['制备技术', '单晶生长技术'],
  'CN112863620A': ['制备技术', '单晶生长技术'],
  'CN112859771A': ['制备技术', '单晶生长技术'],
  'CN112853470A': ['制备技术', '单晶生长技术'],
  'CN117867636A': ['制备技术', '单晶生长技术'],
  'CN214458440U': ['制备技术', '单晶生长技术'],
  'CN214458441U': ['制备技术', '单晶生长技术'],
  'CN117626408A': ['制备技术', '单晶生长技术'],
  'CN113957519B': ['制备技术', '单晶生长技术'],
  'CN114959868A': ['制备技术', '单晶生长技术'],
  'CN114808118A': ['制备技术', '外延生长、薄膜制备技术'],
  'CN114855269A': ['制备技术', '外延生长、薄膜制备技术'],
  'CN114908418A': ['制备技术', '外延生长、薄膜制备技术'],
  'CN114908419

In [ ]:
applicant_rank # 专利数量、专利主题
analysis_result

#### 画图

In [109]:
import os
import matplotlib.pyplot as plt
import matplotlib

def heatmap_visualization(save_dir,analysis_result):
    result_dict = {r["company_name"]: r["分类结果"] for r in analysis_result}
    companies = list(result_dict.keys())
    secondary_types = set()

    for company, patents in result_dict.items():
        for _, types in patents.items():
            if len(types) > 1:
                secondary_types.add(f"{types[0]}_{types[1]}")

    secondary_types = sorted(list(secondary_types))

    # 创建数据矩阵
    data = np.zeros((len(companies), len(secondary_types)))

    # 填充数据
    for i, company in enumerate(companies):
        for _, types in result_dict[company].items():
            if len(types) > 1:
                j = secondary_types.index(f"{types[0]}_{types[1]}")
                data[i, j] += 1

    # 创建热力图
    fig, ax = plt.subplots(figsize=(14, 8))

    # 使用蓝色调色板
    cmap = plt.cm.Blues

    # 创建热力图
    im = ax.imshow(data, cmap=cmap)

    # 添加颜色条
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("专利数量", rotation=-90, va="bottom", fontsize=12)

    # 设置坐标轴刻度和标签
    ax.set_xticks(np.arange(len(secondary_types)))
    ax.set_yticks(np.arange(len(companies)))
    ax.set_xticklabels(secondary_types)
    ax.set_yticklabels(companies)

    # 旋转X轴标签
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # 在每个单元格中添加文本
    for i in range(len(companies)):
        for j in range(len(secondary_types)):
            if data[i, j] > 0:  # 只显示非零值
                text = ax.text(j, i, int(data[i, j]),
                                ha="center", va="center", 
                                color="black" if data[i, j] < np.max(data)/2 else "white")

    # 添加网格线
    ax.set_xticks(np.arange(len(secondary_types)+1)-.5, minor=True)
    ax.set_yticks(np.arange(len(companies)+1)-.5, minor=True)
    ax.grid(which="minor", color="gray", linestyle='-', linewidth=0.5)

    # 设置标题
    ax.set_title("专利二级分类分布热力图", fontsize=16)

    # 调整布局并保存
    fig.tight_layout()
    output_path = os.path.join(save_dir, '专利主体-专利技术热力图.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    return './专利主体-专利技术热力图.png'

def bar_visualization(save_dir,applicant_rank):
    # 分离数据
    names = [f"{item[0]}({item[1]})" for item in applicant_rank]
    counts = [item[2] for item in applicant_rank]

    # 绘制条形图
    plt.figure(figsize=(10, 6))
    plt.barh(names, counts, color='skyblue')
    plt.xlabel('申请数量')
    plt.title('专利申请人及其申请数量')
    plt.gca().invert_yaxis()  # 反转y轴，使得申请数量最多的在顶部

    # 保存到output文件夹
    output_path = os.path.join(save_dir, '专利主体-专利数量柱状图.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()  # 关闭图形，释放内存
    return './专利主体-专利数量柱状图.png'

def visualization(save_dir,applicant_rank,analysis_result):
    # 确保output文件夹存在
    os.makedirs(save_dir, exist_ok=True)

    # 设置中文字体
    matplotlib.rcParams['font.sans-serif'] = ['SimHei']
    matplotlib.rcParams['axes.unicode_minus'] = False
    bar_dir = bar_visualization(save_dir,applicant_rank)
    heatmap_dir = heatmap_visualization(save_dir,analysis_result)
    return bar_dir,heatmap_dir

save_dir = r"D:\GoodStudy\AI_Agents_Hackathon\awesome_patent_mining\patent_analysis_agent\research_agent\output"
bar_dir,heatmap_dir = visualization(save_dir,applicant_rank,analysis_result)

#### 转变成json

In [107]:
import pandas as pd
from collections import defaultdict
# 专利-技术——json格式
def data_to_json(analysis_result,applicant_rank):
    result_dict = {r["company_name"]: r["分类结果"] for r in analysis_result}
    # 创建技术分类统计字典
    tech_stats = defaultdict(lambda: defaultdict(int))

    # 遍历每家公司
    for company, patents in result_dict.items():
        for patent_id, categories in patents.items():
            primary_cat = categories[0]  # 一级分类
            secondary_cat = categories[1]  # 二级分类
            tech_stats[company][primary_cat] += 1
            tech_stats[company][f"{primary_cat}-{secondary_cat}"] += 1

    # 转换为DataFrame并确保所有数值为Python原生int
    df = pd.DataFrame.from_dict(tech_stats, orient='index')
    df = df.fillna(0).applymap(lambda x: int(x))  # 关键修改点

    # 获取所有二级分类并按字母顺序排序
    secondary_categories = sorted([col for col in df.columns if '-' in col])
    df = df[secondary_categories]

    # 转换为结构化的JSON格式
    def df_to_structured_json(df):
        result = {
            "metadata": {
                "data_type": "企业专利技术分布",
                "generated_at": pd.Timestamp.now().strftime("%Y-%m-%d"),
                "companies_count": len(df),
                "technology_categories_count": len(df.columns)
            },
            "companies": []
        }
        
        for company in df.index:
            company_data = {
                "company_name": company,
                "patent_counts": {
                    "by_category": {k: int(v) for k, v in df.loc[company].items()},  # 确保字典值为int
                }
            }
            result["companies"].append(company_data)
        
        return result

    # 转换为字典列表
    json_data = [
        {
            "当前申请(专利权)人": company,
            "专利受理局及专利数量": country,
            "专利总数量": patent_count
        }
        for company, country, patent_count in applicant_rank
    ]

    # 输出JSON
    import json
    applicant_rank_json = json.dumps(json_data, ensure_ascii=False, indent=2)
    return df_to_structured_json(df),applicant_rank_json
company_tech_json,applicant_rank_json = data_to_json(analysis_result,applicant_rank)

C:\Users\17981\AppData\Local\Temp\ipykernel_19924\2627513210.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: int(x))  # 关键修改点


In [128]:
print(applicant_rank_json)

[
  {
    "当前申请(专利权)人": "杭州富加镓业科技有限公司",
    "专利受理局及专利数量": "中国(32), 日本(2), 美国(1)",
    "专利总数量": 35
  },
  {
    "当前申请(专利权)人": "信越化学工业株式会社",
    "专利受理局及专利数量": "韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)",
    "专利总数量": 29
  },
  {
    "当前申请(专利权)人": "日本碍子株式会社",
    "专利受理局及专利数量": "日本(10), 美国(2), 世界知识产权组织(4), 中国(7)",
    "专利总数量": 23
  },
  {
    "当前申请(专利权)人": "中国电子科技集团公司第四十六研究所",
    "专利受理局及专利数量": "中国(18)",
    "专利总数量": 18
  },
  {
    "当前申请(专利权)人": "山东大学",
    "专利受理局及专利数量": "中国(17), 日本(1)",
    "专利总数量": 18
  }
]


In [ ]:
analysis_result

In [127]:
company_tech_json,applicant_rank_json

({'metadata': {'data_type': '企业专利技术分布',
   'generated_at': '2025-04-20',
   'companies_count': 5,
   'technology_categories_count': 6},
  'companies': [{'company_name': '杭州富加镓业科技有限公司',
    'patent_counts': {'by_category': {'制备技术-单晶生长技术': 30,
      '制备技术-外延生长、薄膜制备技术': 5,
      '制备技术-衬底加工技术': 0,
      '器件及应用-功率电子器件': 0,
      '器件及应用-日盲紫外光电探测器': 0,
      '器件及应用-红外焦平面阵列': 0}}},
   {'company_name': '信越化学工业株式会社',
    'patent_counts': {'by_category': {'制备技术-单晶生长技术': 0,
      '制备技术-外延生长、薄膜制备技术': 28,
      '制备技术-衬底加工技术': 1,
      '器件及应用-功率电子器件': 0,
      '器件及应用-日盲紫外光电探测器': 0,
      '器件及应用-红外焦平面阵列': 0}}},
   {'company_name': '日本碍子株式会社',
    'patent_counts': {'by_category': {'制备技术-单晶生长技术': 1,
      '制备技术-外延生长、薄膜制备技术': 16,
      '制备技术-衬底加工技术': 6,
      '器件及应用-功率电子器件': 0,
      '器件及应用-日盲紫外光电探测器': 0,
      '器件及应用-红外焦平面阵列': 0}}},
   {'company_name': '中国电子科技集团公司第四十六研究所',
    'patent_counts': {'by_category': {'制备技术-单晶生长技术': 12,
      '制备技术-外延生长、薄膜制备技术': 3,
      '制备技术-衬底加工技术': 3,
      '器件及应用-功率电子器件': 

In [94]:
patent_miner

['杭州富加镓业科技有限公司的专利主要集中在氧化镓单晶生长技术及外延薄膜制备技术领域。核心技术方向包括：1) 激光辅助加热技术，用于精确控制晶体生长过程中的温度梯度（如CN113913924A、CN114086244A）；2) 籽晶杆及模具结构优化，解决挥发物吸附和熔体输送问题（如CN215976140U、CN214458441U）；3) 热场结构设计，通过分层拼接式保温层释放热应力（如CN215976139U、CN112877770A）；4) 外延生长中的激光加热技术，实现衬底均匀受热（如CN114908418A、CN114855269A）；5) 人工智能辅助的质量预测系统，优化晶体生长参数（如CN112853470A、CN112859771A）。\n\n典型案例分析：\n1) CN113913924A通过激光辅助加热器精准消除固液界面过冷现象，解决了导模法生长中的杂晶问题；\n2) CN215976140U采用螺旋式籽晶杆设计，有效吸附挥发物，将晶体生长成功率提升30%以上；\n3) CN114908418A开发了激光加热HVPE设备，使外延薄膜厚度均匀性达到±2%；\n4) CN215976139U的多层拼接式热场结构，将热场开裂率从15%降至1%以下；\n5) CN112853470A的深度学习预测系统，可实现晶体质量参数的提前30分钟预测，准确率达92%。',
 '信越化学工业株式会社的专利主要集中在氧化镓半导体膜的制备技术，特别是通过雾化CVD法进行外延生长和薄膜制备。核心技术方向包括：1) 提高膜厚的面内均匀性和成膜速度（如JP6925548B1、JP7075525B2）；2) 改善结晶性和表面平滑性（如CN217691180U、JP7090052B2）；3) 降低晶体缺陷和提高半导体特性（如KR1020240087704A、JP7161456B2）。技术问题解决方面，针对工业上大规模生产的需求，提出了多种解决方案，如通过优化载气流量和温度控制成膜速度（JP7075525B2），以及使用特定基板（如钽酸锂晶体基板）提高热稳定性和电特性（JP7161456B2）。典型案例包括JP6925548B1，通过雾化CVD法和整流工序显著提高了膜厚的均匀性和成膜速度；JP7090052B2通过控制基板加热和原料溶液接触方式，实现了表面平滑性极高的氧化镓膜；KR10

In [110]:
patent_miner = [a["综合技术挖掘"] for a in analysis_result]

In [111]:
company_info = [s[0] if isinstance(s,tuple) else s for s in search_result ]

In [101]:
prompt_survey = [{"role": "user", 
                "content": f"""
            你是一个资深的专利分析专家，你需要根据输入的数据形成一版专利申请人分析报告
            现有一下输入：
                **专利申请人排名分析**
                    1.专利申请人排名json数据
                        {applicant_rank_json}
                    2.专利申请人排名：专利主体-专利数量柱状图存储路径：{bar_dir}
                **专利申请人-技术分布分析**
                    1.专利申请人技术分布json数据：
                        {company_tech_json}
                    2. 专利申请人技术分布：专利主题-专利技术热力图存储路径：{heatmap_dir}
                    3. 专利申请人背景信息：{company_info}
                    4. 专利申请人技术分布挖掘：{patent_miner}
            任务：
                1.请整合以上输入数据，形成专利申请人分析，输出格式要是Markdown格式
                2.请将生成的相关图形，插入到Markdown中合适的位置
                3.严格遵循输入的数据，不准臆测
                4.确保信息没有遗漏
            输出格式示例：
            ## 专利申请人分析
            ### 专利申请人排名分析
                在这里整合**专利申请人排名分析**的数据
            ### 专利申请人技术分布分析
                在这里整合**专利申请人-技术分布分析**的数据
            """}]
survey_result = await deepseek_llm.completion(prompt_survey)
print(survey_result)

{'model': 'deepseek-chat', 'messages': [{'role': 'user', 'content': '\n            你是一个资深的专利分析专家，你需要根据输入的数据形成一版专利申请人分析报告\n            现有一下输入：\n                **专利申请人排名分析**\n                    1.专利申请人排名json数据\n                        [\n  {\n    "当前申请(专利权)人": "杭州富加镓业科技有限公司",\n    "专利受理局及专利数量": "中国(32), 日本(2), 美国(1)",\n    "专利总数量": 35\n  },\n  {\n    "当前申请(专利权)人": "信越化学工业株式会社",\n    "专利受理局及专利数量": "韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)",\n    "专利总数量": 29\n  },\n  {\n    "当前申请(专利权)人": "日本碍子株式会社",\n    "专利受理局及专利数量": "日本(10), 美国(2), 世界知识产权组织(4), 中国(7)",\n    "专利总数量": 23\n  },\n  {\n    "当前申请(专利权)人": "中国电子科技集团公司第四十六研究所",\n    "专利受理局及专利数量": "中国(18)",\n    "专利总数量": 18\n  },\n  {\n    "当前申请(专利权)人": "山东大学",\n    "专利受理局及专利数量": "中国(17), 日本(1)",\n    "专利总数量": 18\n  }\n]\n                    2.专利申请人排名：专利主体-专利数量柱状图存储路径：./专利主体-专利数量柱状图.png\n                **专利申请人-技术分布分析**\n                    1.专利申请人技术分布json数据：\n                        {\'metadata\': {\'data_type\': \'企业专利技术分布\', \'genera

In [ ]:
# 分析
prompt_survey = [{"role": "user", 
                "content": f"""
**角色:** 你是一位资深的专利分析专家。

**任务:** 根据以下输入数据，撰写一份详细的专利申请人分析报告。报告需要整合所提供的信息，以清晰、专业的 Markdown 格式呈现，并包含从JSON数据生成的表格和图表占位符。

**输入数据:**

1.  **专利申请人排名分析:**
    * **排名JSON数据:** `{applicant_rank_json}`
        * *说明:* 此JSON包含按专利申请数量排序的申请人列表及其对应的专利数量。
    * **排名柱状图路径 (占位符):** `{bar_dir}`
        * *说明:* 此路径指向一个展示主要申请人及其专利数量对比的柱状图图片。

2.  **专利申请人-技术分布分析:**
    * **技术分布JSON数据:** `{company_tech_json}`
        * *说明:* 此JSON包含主要申请人在不同技术分类下的专利数量分布。
    * **技术分布热力图路径 (占位符):** `{heatmap_dir}`
        * *说明:* 此路径指向一个展示申请人与技术领域之间专利密度关系的热力图图片。
    * **申请人背景信息:** `{company_info}`
        * *说明:* 提供关键申请人的背景资料。
    * **技术分布挖掘:** `{patent_miner}`
        * *说明:* 提供基于专利数据得出的关于申请人技术布局特点、重点研发领域或技术演进趋势的文本分析结论。

**报告生成要求:**

1.  **报告结构:** 严格按照以下 Markdown 结构组织报告：
    ## 专利申请人分析

    ### 专利申请人排名分析
    (此处内容)

    ### 专利申请人技术分布分析
    (此处内容)

2.  **内容生成 - 专利申请人排名分析:**
    * **分析总结:** 基于 `{applicant_rank_json}` 数据，撰写一段分析文字，总结专利申请的整体集中度，并指出排名靠前的核心申请人。
    * **插入图表:** 在分析文字之后，使用 Markdown 语法插入柱状图的占位符：`![专利申请人排名柱状图]({bar_dir})`。并在图表下方添加简短文字说明，例如：“图1：主要专利申请人专利申请数量排名”。

3.  **内容生成 - 专利申请人技术分布分析:**
    * **分析总结:** 
        - 分析 `{company_tech_json}`
        - 整合`{company_info}` 和 `{patent_miner}`的信息。
    * **插入图表:** 在分析文字之后，使用 Markdown 语法插入热力图的占位符：`![专利申请人技术分布热力图]({heatmap_dir})`。并在图表下方添加简短文字说明，例如：“图2：主要专利申请人技术领域分布热力图”。

4.  **格式与约束:**
    * **输出格式:** 必须是完整的 Markdown 格式。
    * **数据驱动:** 所有分析和结论必须严格基于提供的输入数据，禁止添加任何未包含在输入数据中的信息或进行主观臆测。
    * **专业性:** 使用专业、客观、简洁的语言。
    * **图表处理:** `{bar_dir}` 和 `{heatmap_dir}` 仅作为图片路径占位符使用，按要求插入 Markdown 图片链接即可。
    * **表格清晰:** 确保生成的 Markdown 表格格式正确、对齐，易于阅读。

**最终输出:** 一份结构清晰、包含分析、表格和图表占位符的 Markdown 格式专利申请人分析报告。

""" }]
survey_result = await deepseek_llm.completion(prompt_survey)
survey_result

{'model': 'deepseek-chat', 'messages': [{'role': 'user', 'content': '\n**角色:** 你是一位资深的专利分析专家。\n\n**任务:** 根据以下输入数据，撰写一份详细的专利申请人分析报告。报告需要整合所提供的信息，以清晰、专业的 Markdown 格式呈现，并包含从JSON数据生成的表格和图表占位符。\n\n**输入数据:**\n\n1.  **专利申请人排名分析:**\n    * **排名JSON数据:** `[\n  {\n    "公司": "杭州富加镓业科技有限公司",\n    "国家": "中国",\n    "专利数量": 32\n  },\n  {\n    "公司": "中国电子科技集团公司第四十六研究所",\n    "国家": "中国",\n    "专利数量": 18\n  },\n  {\n    "公司": "山东大学",\n    "国家": "中国",\n    "专利数量": 17\n  },\n  {\n    "公司": "西安电子科技大学",\n    "国家": "中国",\n    "专利数量": 16\n  },\n  {\n    "公司": "北京铭镓半导体有限公司",\n    "国家": "中国",\n    "专利数量": 15\n  }\n]`\n        * *说明:* 此JSON包含按专利申请数量排序的申请人列表及其对应的专利数量。\n    * **排名柱状图路径 (占位符):** `专利主题-专利数量柱状图已保存至: \'./专利主题-专利数量柱状图.png\'`\n        * *说明:* 此路径指向一个展示主要申请人及其专利数量对比的柱状图图片。\n\n2.  **专利申请人-技术分布分析:**\n    * **技术分布JSON数据:** `{\'metadata\': {\'data_type\': \'企业专利技术分布\', \'generated_at\': \'2025-04-19\', \'companies_count\': 5, \'technology_categories_count\': 7}, \'companies\': [{\'company_name\': \'杭州富加镓业科技有限

'```markdown\n## 专利申请人分析\n\n### 专利申请人排名分析\n\n根据提供的专利数据，中国企业在氧化镓半导体材料领域的专利申请呈现高度集中态势。排名前五的申请人全部来自中国，专利总量占比达98件（前五名总和），其中杭州富加镓业科技有限公司以32件专利显著领先，其专利数量超过第二名（中国电子科技集团公司第四十六研究所，18件）近80%，展现出技术研发的绝对优势地位。第二梯队由研究所和高校构成，专利数量在15-18件区间，其中山东大学（17件）与西安电子科技大学（16件）的申请量相近，北京铭镓半导体有限公司（15件）紧随其后。这种分布特征表明行业技术研发呈现"一超多强"格局，企业主体与科研院所共同推动技术创新。\n\n![专利申请人排名柱状图](./专利主题-专利数量柱状图.png)  \n*图1：主要专利申请人专利申请数量排名*\n\n### 专利申请人技术分布分析\n\n#### 技术领域分布特征\n通过热力图数据可见，各申请人的技术布局呈现显著差异化：\n1. **杭州富加镓业科技**：技术高度集中于单晶生长技术（30件），占其专利总量的93.8%，核心解决导模法工艺中的热应力控制、挥发物吸附等关键问题，典型如CN112831832A的多狭缝模具设计专利。\n2. **中国电科46所**：形成单晶生长（12件）与衬底加工（5件）的双技术支柱，其CN117238753B专利通过氢气微刻蚀技术提升衬底表面质量。\n3. **山东大学**：唯一在器件应用领域全面布局的机构，日盲紫外探测器专利（6件）领先，其CN116344660A专利实现9780A/W的超高响应度。\n4. **西安电子科大**：外延生长技术（10件）和功率器件（5件）并重，专利CN117558758A通过屏蔽栅结构提升器件耐压能力。\n5. **北京铭镓半导体**：聚焦单晶生长（9件）与外延技术（4件），其CN117888183B专利创新性结合提拉法与导模法。\n\n#### 技术路线对比\n- **制备技术**：企业主体（富加镓业、铭镓半导体）更专注单晶生长产业化技术，而科研机构（山大、西电）在外延生长和器件制备领域更具优势。\n- **应用领域**：仅高校体系（山大、西电）涉足功率器件和探测器开发，企业暂未进入下游器件领域，反映产学研分工特征。\n\n![专利申请人技术分布

In [100]:
company_info,patent_miner

(['杭州富加镓业科技有限公司专注于第四代半导体材料氧化镓的研发、生产和销售。在单晶生长技术方面，公司已成功突破6英寸氧化镓单晶生长技术，并稳定供应4英寸氧化镓晶圆。在外延生长、薄膜制备技术方面，公司利用MOCVD技术，在氧化镓单晶衬底上同质外延出厚度超过10微米的薄膜，经国家权威检测机构验证，该技术已达到国际领先水平。',
  '信越化学工业株式会社是一家日本企业，专注于半导体材料的生产和销售。在薄膜制备技术方面，信越化学工业株式会社拥有丰富的经验和技术积累，特别是在外延生长和衬底加工技术方面。公司通过不断研发和创新，提供高品质的半导体材料，满足市场需求。',
  '日本碍子株式会社在单晶生长技术、外延生长和薄膜制备技术以及衬底加工技术方面有着广泛的技术布局和发展。在单晶生长技术方面，该公司专注于III族氮化物单晶的制造，采用气相生长法和助熔剂法等技术。在外延生长和薄膜制备技术方面，日本碍子株式会社致力于SiC单晶的生长方法，特别是在4H-SiC单晶衬底上的外延生长。此外，该公司还开发了直接自固体的单晶生长技术。在衬底加工技术方面，日本碍子株式会社专注于半导体元件用外延基板的制造，包括沟道层、势垒层、扩散防止层和上覆层的层叠形成。这些技术布局和发展使日本碍子株式会社在半导体领域具有竞争力。',
  '中国电子科技集团公司第四十六研究所（简称中国电科46所）是我国最早从事半导体材料和光纤研究开发与生产的单位之一。在单晶生长技术方面，中国电科46所拥有自主研发的VGF法锗单晶生长技术，并成功制备出高质量的4英寸氧化镓单晶。在外延生长、薄膜制备技术方面，中国电科46所通过综合设计硅外延片所用硅衬底片放置区域的径向温度梯度差值、硅外延层生长温度等参数，提高了大尺寸厚膜硅外延片的晶体质量。在衬底加工技术方面，中国电科46所成功解决了4英寸超薄锗单晶片的切割、研磨、抛光、清洗、测试和封装等关键技术，建成了4英寸超薄锗单晶抛光片研制生产线。',
  '山东大学在上述领域技术布局全面，发展迅速。外延生长、薄膜制备技术领先，支撑日盲紫外光电探测器研发；单晶生长技术突破，提升红外焦平面阵列性能；功率电子器件研究深入，推动产业升级。多学科交叉融合，成果显著，为国家战略需求和产业创新贡献力量。'],
 ['杭州富加镓业科技有限公司的专利主要集中在氧化镓单晶生长技术及外延薄膜制备技术领域。

In [ ]:
# 整合
prompt_survey = [{"role": "user", 
                "content": f"""
**角色:** 资深专利分析专家。

**任务:** 生成 Markdown 格式的专利申请人分析报告。

**核心要求:**
1.  **分析JSON:** 分析 `{applicant_rank_json}` 和 `{company_tech_json}`，生成总结。
2.  **整合信息:**  逐一整合`{company_info}` 和 `{patent_miner}` 。
3.  **整合图表:** 插入图表占位符 `{bar_dir}` 和 `{heatmap_dir}`。
4.  **遵循规范:** 严格基于数据，语言专业，格式标准。

**输入数据:**
* `{applicant_rank_json}`: 排名数据 (需分析)
* `{bar_dir}`: 排名柱状图路径 (占位符)
* `{company_tech_json}`: 技术分布数据 (需分析)
* `{company_info}`: 申请人背景 (需融合)
* `{patent_miner}`: 技术挖掘结论 (需融合)
* `{heatmap_dir}`: 技术热力图路径 (占位符)

**输出示例:**
## 专利申请人分析

### 专利申请人排名分析
在本项目关注的汽车用帘式气囊气体发生器研制关键技术领域内，专利申请量排名靠前的申请人中，主要是中国、日本、瑞典、德国和韩国的创新主体。中国的创新主体主要有均胜汽车安全系统公司，其作为全球领先的汽车安全供应商，专利申请量排名第一，此外还有湖北航天化学技术研究所、湖北航鹏化学动力公司、比亚迪汽车公司、东方久乐汽车安全气囊公司、延峰汽车科技公司等；日本的创新主体主要是大赛璐公司和日本化药公司；德国创新主体主要为采埃孚公司；韩国创新主体以现代摩比斯公司为主；此外还有瑞典的奥托立夫公司。

### 专利申请人技术分布分析
选取7个具有代表性的专利创新主体，将其在汽车用帘式气囊气体发生器研制技术领域的成果分布情况进行如上呈现。可看出，均胜汽车安全系统公司的研究涉及气体发生器总体结构设计技术和高压气体密封存贮技术；大赛璐公司涉及的专利技术相对较丰富，其产出的专利成果在气体发生器总体结构设计技术、高压气体密封存贮技术和高燃速/高热量/低吸湿性产气剂配方技术三个技术方向上均有涉及；奥托立夫公司和采埃孚公司的专利在该领域三个技术上也均有分布，但这两个公司更侧重气体发生器总体结构设计技术研究；湖北航天化学技术研究所和湖北航鹏化学动力公司的研究则重视高燃速、高热量、低吸湿性产气剂配方技术的创新；现代摩比斯公司在气体发生器总体结构设计技术上的创新研究居多。
均胜汽车安全系统公司是一家全球领先的汽车安全供应企业，其在全球范围内拥有广泛的业务布局，该公司通过一系列战略收购与并购活动，对日本高田公司等国外知名汽车安全系统企业进行整合，不断扩大其研发团队和先进技术资源，通过持续的研发和积累，均胜公司在车用气囊气体发生器研制方面经验愈发丰富，创新成果不断推出，专利申请量排位第一。其专利主要通过对点火组件、药剂存储装置、密封防潮结构、排气组件、过滤系统等进行优化设计，提升了气囊的充气速度、稳定性及安全性等。如在专利CN210591781U中提出了一种混合式气体发生器排气装置，有效避免了密封膜在气流作用下脱落，及时扩大排气孔孔径，恢复气流通路，确保安全气囊的正常工作；在专利CN116160991A中提出了一种新型气帘用混合式气体发生器，有效防止传火药吸湿受潮，保证气体发生器的功能正常，确保气帘的安全性和可靠性；在专利CN217259932U中公开了一种免焊接型安全气囊用气体发生器，降低了生产成本，简化了结构，提高了产品的安全性和密封性能。

请根据以上要求和数据生成报告，按照要求生成数据，不要生成无关的内容。
""" }]
survey_result = await deepseek_llm.completion(prompt_survey)
print(survey_result)

In [76]:
patent_miner

['杭州富加镓业科技有限公司的专利主要集中在氧化镓单晶的制备技术领域，特别是导模法生长技术。核心技术方向包括：1) 导模法生长氧化镓单晶的模具设计，如CN112831832A和CN214458440U中提出的多块隔板模具，通过多条狭缝提高表面张力，确保厚尺寸单晶生长的稳定；2) 晶体生长过程中的监控与调节，如CN113026103B中的监控系统，通过压力传感器和红外摄像机实时获取晶体重量和温度，及时调节加热功率；3) 热场结构优化，如CN215976139U和CN117626408A中的分层式堆叠和拼接设计，有效释放高温下的热应力，避免热场结构开裂；4) 激光辅助技术，如CN113957529A和CN113913924A中的激光辅助加热器，用于去除杂晶和挥发物，提高晶体质量；5) 外延生长技术，如CN117976518A中的氧化镓薄膜外延方法，通过纳米孔结构缓冲层实现高质量外延生长。这些专利针对氧化镓单晶生长过程中的技术痛点，如热应力控制、杂晶产生、挥发物附着等，提出了创新的解决方案，显著提高了晶体质量和生产效率。',
 '中国电子科技集团公司第四十六研究所在氧化镓单晶生长、衬底加工和外延生长技术方面具有显著的技术积累。核心技术方向包括：1) 氧化镓单晶生长技术，通过改进模具设计、热场结构和原料处理方法，提高晶体质量和尺寸（如CN117552091A、CN117552087A、CN117552089A）；2) 衬底预处理和加工技术，通过氧气辅助氢气微刻蚀和边缘磨削方法，提高衬底表面质量和防止解理（如CN117238753B、CN114161258A）；3) 外延生长技术，通过改进镓舟结构和原料掺杂方法，降低外延膜中的杂质含量和提高熔体流动性（如CN112210828B、CN114574965B）。典型案例包括：CN117552091A通过模具支撑板设计优化温度分布，实现大尺寸氧化镓晶体的均匀生长；CN117238753B通过氧气辅助氢气微刻蚀，去除加工残留物并提高外延膜质量；CN112210828B通过多层镓舟结构防止石英管腐蚀，降低外延膜中的Si杂质含量。',
 '山东大学的专利主要集中在氧化镓（Ga2O3）和镓基材料的制备技术及其在光电探测器中的应用。核心技术方向包括：1) 氧化镓单晶生长技术，如导模法、水平法、光浮区法等；2) 氧化镓薄膜的外延生长技术，如低压

In [ ]:
# 整合
prompt_survey = [{"role": "user", 
                "content": f"""
**角色:** 你是一位资深的专利分析专家。

**任务:** 
- 根据以下输入数据，撰写一份专利申请人分析报告。
- 核心任务是分析 `applicant_rank_json` 和 `company_tech_json` 数据，并据此生成总结文字和Markdown表格。
- 对于 `company_info` 和 `patent_miner` 的文本信息，要求将其内容**与 `company_tech_json` 的分析结果进行有机融合**，以增强分析的深度、提供背景支撑，并确保整体叙述流畅自然，信息无损。
- 报告需以清晰、专业的 Markdown 格式呈现。

**输入数据:**

1.  **专利申请人排名分析:**
    * **排名JSON数据:** `{applicant_rank_json}`
        * *说明:* 包含申请人及其专利数量的排名。**需要分析此数据，撰写总结，并生成排名表格。**
    * **排名柱状图路径:** `{bar_dir}`
        * *说明:* 指向排名柱状图。**作为占位符直接插入。**

2.  **专利申请人-技术分布分析:**
    * **技术分布JSON数据:** `{company_tech_json}`
        * *说明:* 包含主要申请人在不同技术分类下的专利分布。**需要分析此数据，识别主要技术领域和分布模式，作为本部分分析的基础骨架。**
    * **申请人背景信息:** `{company_info}`
        * *说明:* 关键申请人的背景资料。**提取其中的相关信息点（如主营业务、战略方向），用于解释或印证 `company_tech_json` 数据所反映的技术布局现象。**
    * **技术分布挖掘:** `{patent_miner}`
        * *说明:* 技术布局的挖掘结论。**提取其中的具体洞察（如特定技术热点、趋势），用于深化或细化 `company_tech_json` 数据所反映的分析。**
    * **技术分布热力图路径:** `{heatmap_dir}`
        * *说明:* 指向技术分布热力图。**作为占位符直接插入。**

**报告生成要求:**

1.  **报告结构:** 严格按照以下 Markdown 结构：
    ```markdown
    ## 专利申请人分析

    ### 专利申请人排名分析
    在本项目关注的汽车用帘式气囊气体发生器研制关键技术领域内，专利申请量排名靠前的申请人中，主要是中国、日本、瑞典、德国和韩国的创新主体。中国的创新主体主要有均胜汽车安全系统公司，其作为全球领先的汽车安全供应商，专利申请量排名第一，此外还有湖北航天化学技术研究所、湖北航鹏化学动力公司、比亚迪汽车公司、东方久乐汽车安全气囊公司、延峰汽车科技公司等；日本的创新主体主要是大赛璐公司和日本化药公司；德国创新主体主要为采埃孚公司；韩国创新主体以现代摩比斯公司为主；此外还有瑞典的奥托立夫公司。

    ### 专利申请人技术分布分析
    选取7个具有代表性的专利创新主体，将其在汽车用帘式气囊气体发生器研制技术领域的成果分布情况进行如上呈现。可看出，均胜汽车安全系统公司的研究涉及气体发生器总体结构设计技术和高压气体密封存贮技术；大赛璐公司涉及的专利技术相对较丰富，其产出的专利成果在气体发生器总体结构设计技术、高压气体密封存贮技术和高燃速/高热量/低吸湿性产气剂配方技术三个技术方向上均有涉及；奥托立夫公司和采埃孚公司的专利在该领域三个技术上也均有分布，但这两个公司更侧重气体发生器总体结构设计技术研究；湖北航天化学技术研究所和湖北航鹏化学动力公司的研究则重视高燃速、高热量、低吸湿性产气剂配方技术的创新；现代摩比斯公司在气体发生器总体结构设计技术上的创新研究居多。
    均胜汽车安全系统公司是一家全球领先的汽车安全供应企业，其在全球范围内拥有广泛的业务布局，该公司通过一系列战略收购与并购活动，对日本高田公司等国外知名汽车安全系统企业进行整合，不断扩大其研发团队和先进技术资源，通过持续的研发和积累，均胜公司在车用气囊气体发生器研制方面经验愈发丰富，创新成果不断推出，专利申请量排位第一。其专利主要通过对点火组件、药剂存储装置、密封防潮结构、排气组件、过滤系统等进行优化设计，提升了气囊的充气速度、稳定性及安全性等。如在专利CN210591781U中提出了一种混合式气体发生器排气装置，有效避免了密封膜在气流作用下脱落，及时扩大排气孔孔径，恢复气流通路，确保安全气囊的正常工作；在专利CN116160991A中提出了一种新型气帘用混合式气体发生器，有效防止传火药吸湿受潮，保证气体发生器的功能正常，确保气帘的安全性和可靠性；在专利CN217259932U中公开了一种免焊接型安全气囊用气体发生器，降低了生产成本，简化了结构，提高了产品的安全性和密封性能。

    ```

2.  **内容生成 - 专利申请人排名分析:**
    * **分析与总结 (基于JSON):** 根据 `applicant_rank_json` 数据，撰写一段文字，总结申请人排名情况和主要发现。
    * **插入图表占位符:** 在表格下方插入柱状图占位符：`![专利申请人排名柱状图]({bar_dir})`，并附简短图注。

3.  **内容生成 - 专利申请人技术分布分析:**
    * **融合分析与总结 (基于JSON + 融合文本):**
        * **核心步骤:**
            1.  以 `company_tech_json` 的分析为基础，识别出主要申请人的技术布局特点（如重点领域、技术广度等）。
            2.  **在叙述这些特点时**，查阅 `{company_info}` 和 `{patent_miner}`，找到能够**支撑、解释或深化**这些分析点的信息。
            3.  将这些来自 `company_info`（背景、原因）和 `patent_miner`（具体洞察、趋势）的信息，**用自然的语言、恰当的逻辑关系编织进**对 `company_tech_json` 分析结果的阐述中。
            4.  目标是生成一段**观点清晰、论据充分（结合了JSON数据和文本信息）、语句连贯**的分析段落。避免生硬地插入整段原文。
        * **信息完整性:** 确保在融合过程中，`company_info` 和 `patent_miner` 提供的**核心观点和关键信息**都以适当的方式反映在最终的分析文本中，没有重要遗漏。
    * **插入图表占位符:** 在表格下方插入热力图占位符：`![专利申请人技术分布热力图]({heatmap_dir})`，并附简短图注。

4.  **格式与约束:**
    * **输出格式:** 标准 Markdown。
    * **分析重点:** `applicant_rank_json` 和 `company_tech_json` 是分析基础。
    * **信息融合:** `company_info` 和 `patent_miner` 作为**论据或背景**融入分析，要求**无损信息、语句流畅、逻辑自然**，避免信息孤岛。
    * **专业客观:** 语言专业、简洁。
    * **图表处理:** 图表路径仅作占位符。
    * **数据来源:** 报告内容严格依据输入数据，禁止臆测。

**最终输出:** 一份 Markdown 报告，其中包含对排名JSON的分析；对技术分布JSON的分析，并**自然流畅地融合**了背景信息和挖掘发现作为支撑或解释；以及两个图表的占位符。
""" }]
survey_result = await deepseek_llm.completion(prompt_survey)
print(survey_result)

In [ ]:
company_tech_json

In [112]:
# 整合
prompt_survey = [{"role": "user", 
                "content": f"""
**角色:** 你是一位资深的专利分析专家。
**任务:** 根据以下输入数据，撰写一份专利申请人分析报告。
**输入数据:**
1.  **专利申请人排名分析:**
    * **排名JSON数据 (需分析):** `{applicant_rank_json}`
        * *说明:* 包含申请人及其专利数量的排名。**需要分析此数据，撰写总结，并生成排名表格。**
    * **排名柱状图路径 (直接整合):** `{bar_dir}`
        * *说明:* 指向排名柱状图。**作为占位符直接插入。**
2.  **专利申请人-技术分布分析:**
    * **技术分布JSON数据 (需分析):** `{company_tech_json}`
        * *说明:* 包含主要申请人在不同技术分类下的专利分布。**需要分析此数据，识别主要技术领域和分布模式，作为本部分分析的基础骨架。**
    * **申请人背景信息 (需融合):** `{company_info}`
        * *说明:* 关键申请人的背景资料。**
    * **技术分布挖掘 (需融合):** `{patent_miner}`
        * *说明:* 专利申请人的核心方向、**
    * **技术分布热力图路径 (直接整合):** `{heatmap_dir}`
        * *说明:* 指向技术分布热力图。**作为占位符直接插入。**

**报告生成要求:**

1.  **报告结构:** 严格按照以下 Markdown 结构：
    ```markdown
    ## 专利申请人分析

    ### 专利申请人排名分析

    ### 专利申请人技术分布分析
    (包含：**专利申请人技术分布分析**，**申请人背景信息和技术分布挖掘的整合**)
    ...
    ```

2.  **内容生成:**
    * **专利申请人分析:** 根据 `applicant_rank_json` 数据，撰写一段文字，总结申请人排名情况和主要发现。
    * **专利申请人技术分布分析:** 实现对`company_tech_json` 的分析，识别出主要申请人的技术布局特点。。
    * **插入图表占位符:** 在合适位置插入占位符：`![专利申请人排名柱状图]({bar_dir})`、`![专利申请人技术分布热力图]({heatmap_dir})`并附简短图注。

3.  **申请人背景信息和技术分布挖掘的整合**
    - 主要任务：逐一分析每个专利申请人背景信息、核心技术方向、技术问题解决分析、典型专利说明等。
    - 结合**专利申请人技术分布分析:**，将`company_info`和 `patent_miner`的信息整合，**用专业的语言、恰当的逻辑关系**实现信息的整合。
    - 信息完整性:** 确保在整合过程中，`company_info` 和 `patent_miner` 提供的信息都以适当的方式反映在最终的分析文本中，没有遗漏。
    * **插入图表占位符:** 在合适位置插入热力图占位符：`![专利申请人技术分布热力图]({heatmap_dir})`，并附简短图注。

4.  **格式与约束:**
    * **输出格式:** 标准 Markdown。
    * **信息分析:** `applicant_rank_json` 和 `company_tech_json` 是分析基础。
    * **信息整合:** 结合信息分析，整合`company_info` 和 `patent_miner`，要求**无损信息、语句流畅、逻辑自然**，避免信息孤岛。
    * **专业客观:** 语言专业、简洁。
    * **图表处理:** 图表路径仅作占位符。
    * **数据来源:** 报告内容严格依据输入数据，禁止臆测。

**最终输出:** 一份 Markdown 报告，其中包含对排名JSON的分析；对技术分布JSON的分析，申请人背景信息和技术分布挖掘的整合；以及两个图表的占位符。
""" }]
survey_result = await deepseek_llm.completion(prompt_survey)
print(survey_result)

{'model': 'deepseek-chat', 'messages': [{'role': 'user', 'content': '\n**角色:** 你是一位资深的专利分析专家。\n**任务:** 根据以下输入数据，撰写一份专利申请人分析报告。\n**输入数据:**\n1.  **专利申请人排名分析:**\n    * **排名JSON数据 (需分析):** `[\n  {\n    "当前申请(专利权)人": "杭州富加镓业科技有限公司",\n    "专利受理局及专利数量": "中国(32), 日本(2), 美国(1)",\n    "专利总数量": 35\n  },\n  {\n    "当前申请(专利权)人": "信越化学工业株式会社",\n    "专利受理局及专利数量": "韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)",\n    "专利总数量": 29\n  },\n  {\n    "当前申请(专利权)人": "日本碍子株式会社",\n    "专利受理局及专利数量": "日本(10), 美国(2), 世界知识产权组织(4), 中国(7)",\n    "专利总数量": 23\n  },\n  {\n    "当前申请(专利权)人": "中国电子科技集团公司第四十六研究所",\n    "专利受理局及专利数量": "中国(18)",\n    "专利总数量": 18\n  },\n  {\n    "当前申请(专利权)人": "山东大学",\n    "专利受理局及专利数量": "中国(17), 日本(1)",\n    "专利总数量": 18\n  }\n]`\n        * *说明:* 包含申请人及其专利数量的排名。**需要分析此数据，撰写总结，并生成排名表格。**\n    * **排名柱状图路径 (直接整合):** `./专利主体-专利数量柱状图.png`\n        * *说明:* 指向排名柱状图。**作为占位符直接插入。**\n2.  **专利申请人-技术分布分析:**\n    * **技术分布JSON数据 (需分析):** `{\'metadata\': {\'data_type\': \'企业专利技术分布\', \'generated_at\': \'202

In [125]:
output_path = os.path.join(r"D:\GoodStudy\AI_Agents_Hackathon\awesome_patent_mining\patent_analysis_agent\research_agent\output", 'patent_survey_v5.md')

with open(output_path, "w", encoding="utf-8") as f:
    f.write(survey)

In [ ]:
print(survey_result)

In [118]:
# 整合
prompt_applicant_rank = [{"role": "user", 
                "content": f"""
**角色:** 你是一位资深的专利分析专家。
**任务:** 根据以下输入数据，撰写一份专利申请人分析报告。
**输入数据:**
1.  **专利申请人排名分析:**
    * **排名JSON数据 (需分析):** `{applicant_rank_json}`
        * *说明:* 包含申请人及其专利数量的排名。
    * **排名柱状图路径 (直接整合):** `{bar_dir}`
        * *说明:* 指向排名柱状图。**作为占位符直接插入。**
2.  **专利申请人-技术分布分析:**
    * **技术分布JSON数据 (需分析):** `{company_tech_json}`
        * *说明:* 包含主要申请人在不同技术分类下的专利分布。
    * **技术分布热力图路径 (直接整合):** `{heatmap_dir}`
        * *说明:* 指向技术分布热力图。**作为占位符直接插入。**

2.  **内容生成:**
    * **专利申请人分析:** 根据 `applicant_rank_json` 数据，撰写一段文字，总结申请人排名情况和主要发现。
    * **专利申请人技术分布分析:** 实现对`company_tech_json` 的分析，识别出主要申请人的技术布局特点。。
    * **插入图表占位符:** 在合适位置插入占位符：`![专利申请人排名柱状图]({bar_dir})`、`![专利申请人技术分布热力图]({heatmap_dir})`并附简短图注。
输出格式:
### 专利申请人分析报告
#### 一、专利申请人排名分析
#### 二、专利申请人技术分布
""" }]


In [119]:
# 整合
prompt_applicant_tech = [{"role": "user", 
                "content": f"""
**角色:** 你是一位资深的专利分析专家。
**任务:** 根据以下输入数据，实现申请人背景信息和技术分布挖掘的整合。
**输入数据:**
    **专利申请人-技术分布分析:**
    * **技术分布JSON数据 (需分析):** `{company_tech_json}`
        * *说明:* 包含主要申请人在不同技术分类下的专利分布。**需要分析此数据，识别主要技术领域和分布模式，作为本部分分析的基础骨架。**
    * **申请人背景信息 (需融合):** `{company_info}`
        * *说明:* 关键申请人的背景资料。**
    * **技术分布挖掘 (需融合):** `{patent_miner}`
        * *说明:* 专利申请人的核心方向、针对哪些技术痛点，提出了哪些解决方案，效果指标如何、代表性的专利**

**申请人背景信息和技术分布挖掘的整合**
    - 主要任务：逐一分析整合每个专利申请人背景信息、核心技术方向、技术问题解决分析、典型专利说明等。
    - 结合**技术分布JSON数据:**，将`company_info`和 `patent_miner`的信息整合，**用专业的语言、恰当的逻辑关系**实现信息的整合。
    - 信息完整性:** 确保在整合过程中，`company_info` 和 `patent_miner` 提供的信息都以适当的方式反映在最终的分析文本中，没有遗漏。

**格式与约束:**
* **输出格式:** 标准 Markdown。
* **专业客观:** 语言专业、简洁。
* **数据来源:** 报告内容严格依据输入数据，禁止臆测。

输出格式：
#### 三、专利申请人技术布局分析
""" }]


In [120]:

tasks = [deepseek_llm.completion(prompt_applicant_rank),deepseek_llm.completion(prompt_applicant_tech)]
result = await asyncio.gather(*tasks)

{'model': 'deepseek-chat', 'messages': [{'role': 'user', 'content': '\n**角色:** 你是一位资深的专利分析专家。\n**任务:** 根据以下输入数据，撰写一份专利申请人分析报告。\n**输入数据:**\n1.  **专利申请人排名分析:**\n    * **排名JSON数据 (需分析):** `[\n  {\n    "当前申请(专利权)人": "杭州富加镓业科技有限公司",\n    "专利受理局及专利数量": "中国(32), 日本(2), 美国(1)",\n    "专利总数量": 35\n  },\n  {\n    "当前申请(专利权)人": "信越化学工业株式会社",\n    "专利受理局及专利数量": "韩国(1), 世界知识产权组织(4), 中国(9), 中国台湾(1), 日本(13), 美国(1)",\n    "专利总数量": 29\n  },\n  {\n    "当前申请(专利权)人": "日本碍子株式会社",\n    "专利受理局及专利数量": "日本(10), 美国(2), 世界知识产权组织(4), 中国(7)",\n    "专利总数量": 23\n  },\n  {\n    "当前申请(专利权)人": "中国电子科技集团公司第四十六研究所",\n    "专利受理局及专利数量": "中国(18)",\n    "专利总数量": 18\n  },\n  {\n    "当前申请(专利权)人": "山东大学",\n    "专利受理局及专利数量": "中国(17), 日本(1)",\n    "专利总数量": 18\n  }\n]`\n        * *说明:* 包含申请人及其专利数量的排名。\n    * **排名柱状图路径 (直接整合):** `./专利主体-专利数量柱状图.png`\n        * *说明:* 指向排名柱状图。**作为占位符直接插入。**\n2.  **专利申请人-技术分布分析:**\n    * **技术分布JSON数据 (需分析):** `{\'metadata\': {\'data_type\': \'企业专利技术分布\', \'generated_at\': \'2025-04-20\', \'companies_co

In [124]:
print("\n".join(analysis_result))
survey = "\n".join(analysis_result)

### 专利申请人分析报告

#### 一、专利申请人排名分析  
根据专利数据统计，排名前五的申请人及其专利数量如下：  
1. **杭州富加镓业科技有限公司**以35件专利位居榜首，其中中国专利32件，海外布局覆盖日本（2件）和美国（1件），显示其以中国市场为核心、逐步拓展国际市场的策略。  
2. **信越化学工业株式会社**（29件）专利分布较广，覆盖韩国、世界知识产权组织（WIPO）、中国、中国台湾、日本和美国，体现其全球化布局特点，且日本本土专利（13件）占比最高。  
3. **日本碍子株式会社**（23件）专利同样侧重国际化，WIPO（4件）和中国（7件）是其重要目标市场。  
4. **中国电子科技集团公司第四十六研究所**和**山东大学**并列18件，但前者专利全部集中于中国，后者则在中国（17件）和日本（1件）有少量布局。  

**主要发现：**  
- 中国企业在总量上占据优势（如杭州富加镓业、中国电科46所），但海外布局较弱；日本企业（信越化学、日本碍子）则更注重多国专利保护。  
- 高校（山东大学）的国际化申请量较低，可能以技术研发而非商业化为主导。  

![专利申请人排名柱状图](./专利主体-专利数量柱状图.png)  
*图1：主要专利申请人排名及专利数量分布*  

---  

#### 二、专利申请人技术分布  
从技术领域来看，各申请人的布局差异显著：  
1. **杭州富加镓业科技有限公司**：技术高度集中于**单晶生长技术**（30件），占其总量的85.7%，外延生长技术（5件）为辅，其他领域未见布局，显示其专注于材料制备上游环节。  
2. **信越化学工业株式会社**：以**外延生长、薄膜制备技术**（28件）为核心，占比96.6%，衬底加工技术仅1件，未见器件应用专利，反映其作为材料供应商的定位。  
3. **日本碍子株式会社**：技术分布较均衡，以外延生长（16件）为主，衬底加工（6件）和单晶生长（1件）为辅，但同样未涉足器件领域。  
4. **中国电科46所**：单晶生长（12件）占主导，外延和衬底加工各3件，技术链覆盖较完整但专利总量有限。  
5. **山东大学**：技术多元化特征明显，除单晶生长（6件）和外延技术（4件）外，还布局**日盲紫外光电探测器**（6件）和少量功率电子器件、红外焦平面专